In [1]:
import sys
sys.path.insert(0, '..')

from NtupleDataFormat import HGCalNtuple
import root_numpy as rnp
import root_numpy.tmva as rnptmva
import pandas as pd
import numpy as np
from multiprocessing import Pool
from shutil import copyfile

# The purpose of this file is to demonstrate mainly the objects
# that are in the HGCalNtuple
import ROOT
import os
import array
import socket
import datetime
import optparse
import yaml

import python.l1THistos as histos
import python.clusterTools as clAlgo
import traceback
import subprocess32
from python.utils import debugPrintOut

import python.file_manager as fm
import python.selections as selections


Welcome to JupyROOT 6.14/02
gen_part_selections: 10


In [2]:

maxEvents = 10
input_base_dir = '/eos/cms/store/cmst3/group/l1tr/cerminar/hgcal/CMSSW1015/'
input_sample_dir = 'SingleE_FlatPt-2to100/SingleE_FlatPt-2to100_PU0_v13/181116_090500/0000/'
debug = 0
tree_name = 'hgcalTriggerNtuplizer/HGCalTriggerNtuple'
    
range_ev = (0, maxEvents)
        
    
input_files = fm.get_files_for_processing(input_dir=os.path.join(input_base_dir,
                                                                 input_sample_dir),
                                          tree=tree_name,
                                          nev_toprocess=maxEvents,
                                          debug=debug)

ntuple = HGCalNtuple(input_files, tree=tree_name)
range_ev = (0, ntuple.nevents())

ntuple.setCache(learn_events=1, entry_range=range_ev)
evt_idx = 0

dir already indexed, will read metadata...
[]
Resetting cache: 0
Setting new cache size: 0
Setting # of entries for cache learning: None to 1
Setting cache entry range: 0
Cache size: 400000000


mkdir: illegal option -- -
usage: mkdir [-pv] [-m mode] directory ...
[eos-cp] path=/eos/cms/store/cmst3/group/l1tr/cerminar/hgcal/CMSSW1015/SingleE_FlatPt-2to100/SingleE_FlatPt-2to100_PU0_v13/181116_090500/0000/metadata.json size=39941
[eos-cp] going to copy 1 files and 39.94 kB
[eos-cp] copied 1/1 files and 39.94 kB in 0.40 seconds with 100.93 kB/s


In [3]:
dir(HGCalNtuple)

['PrintCacheStats',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__iter__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'getEvent',
 'nevents',
 'setCache',
 'tree']

In [36]:
event = ntuple.getEvent(evt_idx)
dataframe = event.getDataFrame('tower')
evt_idx+=2

In [140]:
dataframe.sort_values(by='pt', ascending=False)


,pt,energy,eta,phi,etEm,etHad,iEta,iPhi,etLayers,momentum
1079,2.953852,18.755224,-2.53525,-0.305433,2.943721,0.010131,12,32,"[0.0443579, 0.0, 0.0665428, 0.0, 0.0468969, 0....","<ROOT.TLorentzVector object (""TLorentzVector"")..."
81,2.600636,15.191966,2.45075,2.836160,2.592420,0.008215,11,68,"[0.0382426, 0.0, 0.107434, 0.0, 0.431619, 0.0,...","<ROOT.TLorentzVector object (""TLorentzVector"")..."
68,2.435223,15.462238,2.53525,2.836160,2.435223,0.000000,12,68,"[0.0, 0.0, 0.0782983, 0.0, 0.136339, 0.0, 0.84...","<ROOT.TLorentzVector object (""TLorentzVector"")..."
1078,2.243791,14.246757,-2.53525,-0.392699,2.243791,0.000000,12,31,"[0.00234026, 0.0, 0.0568323, 0.0, 0.228434, 0....","<ROOT.TLorentzVector object (""TLorentzVector"")..."
80,0.411163,2.401867,2.45075,2.923426,0.411163,0.000000,11,69,"[0.00213319, 0.0, 0.0228051, 0.0, 0.0192162, 0...","<ROOT.TLorentzVector object (""TLorentzVector"")..."
493,0.206617,1.426182,2.61975,-2.748893,0.206617,0.000000,13,4,"[0.0260708, 0.0, 0.0284844, 0.0, 0.0230024, 0....","<ROOT.TLorentzVector object (""TLorentzVector"")..."
67,0.172438,1.094880,2.53525,2.923426,0.172438,0.000000,12,69,"[0.00643419, 0.0, 0.0349933, 0.0, 0.01327, 0.0...","<ROOT.TLorentzVector object (""TLorentzVector"")..."
1006,0.153357,0.895856,-2.45075,-0.392699,0.153357,0.000000,11,31,"[0.00649954, 0.0, 0.0014212, 0.0, 0.00636737, ...","<ROOT.TLorentzVector object (""TLorentzVector"")..."
1007,0.113874,0.665208,-2.45075,-0.305433,0.113874,0.000000,11,32,"[0.0, 0.0, 0.00143085, 0.0, 0.00502572, 0.0, 0...","<ROOT.TLorentzVector object (""TLorentzVector"")..."
82,0.097017,0.566737,2.45075,2.748893,0.097017,0.000000,11,67,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.017...","<ROOT.TLorentzVector object (""TLorentzVector"")..."


In [38]:
dataframe['momentum'] = ROOT.TLorentzVector()


In [39]:
def fill_momentum(tower):
    vector = ROOT.TLorentzVector()
    vector.SetPtEtaPhiE(tower.pt, tower.eta, tower.phi, tower.energy)
    tower.momentum = vector
    print tower.pt, tower.momentum.Pt()
#     tower.pt = 2
    return tower

In [42]:

dataframe = dataframe.apply(fill_momentum, axis=1)

0.0295318886638 0.0295318886638
0.0295318886638 0.0295318886638
0.0255035944283 0.0255035944283
0.00485661858693 0.00485661858693
0.0168509706855 0.0168509706855
0.0119585348293 0.0119585348293
0.0223738923669 0.0223738923669
0.0127788744867 0.0127788744867
0.0308741591871 0.0308741591871
0.021928679198 0.021928679198
0.0207093171775 0.0207093171775
0.0183384213597 0.0183384213597
0.0158152580261 0.0158152580261
0.0213683992624 0.0213683992624
0.023839795962 0.023839795962
0.0502567961812 0.0502567961812
0.0245974157006 0.0245974157006
0.0214755069464 0.0214755069464
0.042647331953 0.042647331953
0.0275424718857 0.0275424718857
0.0311235263944 0.0311235263944
0.0459956414998 0.0459956414998
0.0217661466449 0.0217661466449
0.0590048208833 0.0590048208833
0.0249168928713 0.0249168928713
0.0317496620119 0.0317496620119
0.0326761044562 0.0326761044562
0.0332131013274 0.0332131013274
0.0316560789943 0.0316560789943
0.0329197347164 0.0329197347164
0.0366231910884 0.0366231910884
0.0171941407

0.0136454813182 0.0136454813182
0.0166571885347 0.0166571885347
0.0294744968414 0.0294744968414
0.0206074491143 0.0206074491143
0.02454489097 0.02454489097
0.0213943235576 0.0213943235576
0.0299854967743 0.0299854967743
0.0319462940097 0.0319462940097
0.0412609800696 0.0412609800696
0.0287338458002 0.0287338458002
0.0227134805173 0.0227134805173
0.0302148237824 0.0302148237824
0.0205210801214 0.0205210801214
0.028331765905 0.028331765905
0.0175060853362 0.0175060853362
0.0310890004039 0.0310890004039
0.00617219414562 0.00617219414562
0.0194348804653 0.0194348804653
0.0167119912803 0.0167119912803
0.0233094152063 0.0233094152063
0.0117377741262 0.0117377741262
0.0285391099751 0.0285391099751
0.044503968209 0.044503968209
0.0117377713323 0.0117377713323
0.0463494770229 0.0463494770229
0.0334448888898 0.0334448888898
0.00681177619845 0.00681177619845
0.0298893991858 0.0298893991858
0.0154286660254 0.0154286660254
0.0297877565026 0.0297877565026
0.0273908060044 0.0273908060044
0.2066165506

0.0405546203256 0.0405546203256
0.0238918252289 0.0238918252289
0.0442798100412 0.0442798100412
0.00235826103017 0.00235826103017
0.0097705675289 0.0097705675289
0.0507191605866 0.0507191605866
0.0152734518051 0.0152734518051
0.0149325216189 0.0149325216189
0.0437751486897 0.0437751486897
0.0224978998303 0.0224978998303
0.0320375338197 0.0320375338197
0.0192008204758 0.0192008204758
0.0532500892878 0.0532500892878
0.0148607743904 0.0148607743904
0.00867114216089 0.00867114216089
0.0177395492792 0.0177395492792
0.0226139817387 0.0226139817387
0.0452951490879 0.0452951490879
0.0310295633972 0.0310295633972
0.0432958267629 0.0432958267629
0.0555643700063 0.0555643700063
0.0361594259739 0.0361594259739
0.0358644351363 0.0358644351363
0.0457123145461 0.0457123145461
0.0163695961237 0.0163695961237
0.0474822372198 0.0474822372198
0.0274430923164 0.0274430923164
0.0207397565246 0.0207397565246
0.0403850562871 0.0403850562871
0.0154857272282 0.0154857272282
0.00907151401043 0.00907151401043
0.

0.0143408644944 0.0143408644944
0.0446883365512 0.0446883365512
0.00267903809436 0.00267903809436
0.00223183631897 0.00223183631897
0.0677683725953 0.0677683725953
0.0487611703575 0.0487611703575
0.0241297539324 0.0241297539324
0.0172064844519 0.0172064844519
0.0326329320669 0.0326329320669
0.032549392432 0.032549392432
0.029279358685 0.029279358685
0.0199280176312 0.0199280176312
0.0141750648618 0.0141750648618
0.0251971874386 0.0251971874386
0.0227889288217 0.0227889288217
0.0247773267329 0.0247773267329
0.0326889567077 0.0326889567077
0.153357028961 0.153357028961
0.113873504102 0.113873504102
0.0288381241262 0.0288381241262
0.0234480854124 0.0234480854124
0.0347364209592 0.0347364209592
0.0286095160991 0.0286095160991
0.0476484671235 0.0476484671235
0.0259223189205 0.0259223189205
0.0363711602986 0.0363711602986
0.037001196295 0.037001196295
0.0229561906308 0.0229561906308
0.0246615577489 0.0246615577489
0.0297269914299 0.0297269914299
0.0379096157849 0.0379096157849
0.024871688336

0.0207129921764 0.0207129921764
0.0587803609669 0.0587803609669
0.0537041388452 0.0537041388452
0.0275483317673 0.0275483317673
0.0271781608462 0.0271781608462
0.0324347987771 0.0324347987771
0.0306162703782 0.0306162703782
0.0299824681133 0.0299824681133
0.0335992276669 0.0335992276669
0.0240425374359 0.0240425374359
0.0276259705424 0.0276259705424
0.0251247491688 0.0251247491688
0.0152755323797 0.0152755323797
0.00218870979734 0.00218870979734
0.0185838602483 0.0185838602483
0.0401475392282 0.0401475392282
0.0136450054124 0.0136450054124
0.0310923624784 0.0310923624784
0.0136450054124 0.0136450054124
0.0243247095495 0.0243247095495
0.0650483891368 0.0650483891368
0.0250608865172 0.0250608865172
0.0625777766109 0.0625777766109
0.0466124862432 0.0466124862432
0.017048843205 0.017048843205
0.0365479663014 0.0365479663014
0.013645007275 0.013645007275
0.0427190884948 0.0427190884948
0.00393534870818 0.00393534870818
0.0259896628559 0.0259896628559
0.0298384372145 0.0298384372145
0.040626

In [47]:
index = 2
dataframe.loc[index].momentum.Pt(), dataframe.loc[index].momentum.Pt(), dataframe.loc[index].momentum.Eta(), dataframe.loc[index].momentum.Phi()

(0.004856618586927652,
 0.004856618586927652,
 2.957750082015986,
 2.923426389694214)

In [43]:
dataframe.iloc[-1].momentum.Pt(), 

0.012979819439351559

In [49]:
dataframe.momentum.sum().Pt()

0.8867246498580591

In [71]:
dataframe.iloc[0]

pt                                                  0.0295319
energy                                               0.285079
eta                                                   2.95775
phi                                                   3.09796
etEm                                               0.00221948
etHad                                               0.0273124
iEta                                                       17
iPhi                                                       71
etLayers    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
momentum    <ROOT.TLorentzVector object ("TLorentzVector")...
Name: 0, dtype: object

In [115]:
test = pd.DataFrame(columns=['pt', 'energy', 'eta', 'phi'], dtype=object)

In [116]:
import math
math.pi

3.141592653589793

In [131]:
test = test.append([{'pt': 20., 'energy': 20., 'phi': 0, 'eta':2.}], ignore_index=True)
test = test.append([{'pt': 30., 'energy': 20., 'phi': math.pi, 'eta':-2.}], ignore_index=True)

In [132]:
test['momentum'] = ROOT.TLorentzVector()
test = test.apply(fill_momentum, axis=1)

10.0 10.0
10.0 10.0
10.0 10.0
20.0 20.0
10.0 10.0
20.0 20.0
30.0 30.0
20.0 20.0
30.0 30.0


In [133]:
test

,pt,energy,eta,phi,momentum
0,10.0,20.0,2.0,0.000000,"<ROOT.TLorentzVector object (""TLorentzVector"")..."
1,10.0,20.0,-2.0,3.141593,"<ROOT.TLorentzVector object (""TLorentzVector"")..."
2,20.0,20.0,2.0,0.000000,"<ROOT.TLorentzVector object (""TLorentzVector"")..."
3,10.0,20.0,-2.0,3.141593,"<ROOT.TLorentzVector object (""TLorentzVector"")..."
4,20.0,20.0,2.0,0.000000,"<ROOT.TLorentzVector object (""TLorentzVector"")..."
5,30.0,20.0,-2.0,3.141593,"<ROOT.TLorentzVector object (""TLorentzVector"")..."
6,20.0,20.0,2.0,0.000000,"<ROOT.TLorentzVector object (""TLorentzVector"")..."
7,30.0,20.0,-2.0,3.141593,"<ROOT.TLorentzVector object (""TLorentzVector"")..."


In [134]:
test.momentum.sum().Pt(), test.momentum.sum().Eta(), test.momentum.sum().Phi()

(10.0, -2.0000000000000004, 3.1415926535897922)

In [200]:

tc_geom_df = pd.DataFrame()
tc_rod_bins = pd.DataFrame()

# read the geometry dump
geom_file = '/eos/cms/store/cmst3/user/cerminar/hgcal/CMSSW932/geom/test_triggergeom.root'

tc_geom_tree = HGCalNtuple([geom_file], tree='hgcaltriggergeomtester/TreeTriggerCells')
print ('read TC GEOM tree with # events: {}'.format(tc_geom_tree.nevents()))


read TC GEOM tree with # events: 1410740


In [201]:
tc_geom_df = convertGeomTreeToDF(tc_geom_tree._tree)
tc_geom_df['radius'] = np.sqrt(tc_geom_df['x']**2+tc_geom_df['y']**2)
tc_geom_df['eta'] = np.arcsinh(tc_geom_df.z/tc_geom_df.radius)


tc_rod_bins = pd.read_csv(filepath_or_buffer='../TCmapping_v2.txt',
                          sep=' ',
                          names=['id', 'rod_x', 'rod_y'],
                          index_col=False)
tc_rod_bins['rod_bin'] = tc_rod_bins.apply(func=lambda cell: (int(cell.rod_x), int(cell.rod_y)), axis=1)

tc_geom_df = pd.merge(tc_geom_df, tc_rod_bins, on='id')


In [673]:
# input_files = listFiles(os.path.join('/eos/cms/store/cmst3/user/cerminar/hgcal/CMSSW932/', 'photonPt35_PU0/NTP/v5/'))
#input_files = listFiles(os.path.join('/eos/cms/store/cmst3/user/cerminar/hgcal/CMSSW932/', 'SinglePiPt5Eta1p6_2p8_PU0/NTP/v4/'))
input_files = listFiles(os.path.join('/eos/cms/store/cmst3/group/l1tr/cerminar/hgcal/CMSSW932/', 'SinglePiPt5Eta1p6_2p8_PU0/NTP/v7/'))

ntuple = HGCalNtuple(input_files, tree='hgcalTriggerNtuplizer/HGCalTriggerNtuple')
nevents = ntuple.nevents()
print '# of events: {}'.format(nevents)

# of events: 6000


In [876]:
evt_idx = 359


In [877]:
#evt_idx +=1

event = ntuple.getEvent(evt_idx)
print ("--- Event {}, @ {}".format(event.entry(), datetime.datetime.now()))
print ('    run: {}, lumi: {}, event: {}'.format(event.run(), event.lumi(), event.event()))

# get the interesting data-frames
genParts = event.getDataFrame(prefix='gen')


branches = [(event, 'genpart'), (event, 'hgcdigi'), (event, 'tc'), (event, 'cl'), (event, 'cl3d'), (event, 'tower')]

dataframes = []

for event, name in branches:
    dataframes.append(event.getDataFrame(name))

genParticles = dataframes[0]
hgcDigis = dataframes[1]
triggerCells = dataframes[2]
triggerClusters = dataframes[3]
trigger3DClusters = dataframes[4]
triggerTowers = dataframes[5]



--- Event 359, @ 2018-05-08 11:46:06.153989
    run: 1, lumi: 15, event: 14360


In [878]:
import copy 
print triggerCells.columns
print triggerClusters.columns
columns = ['id', 'eta', 'phi', 'x', 'y', 'subdet', 'layer', 'pt', 'energy']
columns_tc = copy.copy(columns)
columns_2dcl = copy.copy(columns)
columns_tc.extend(['mipPt','cluster', 'zside'])
columns_2dcl.extend(['cells', 'ncells','rod_bin_max', 'rod_bin_shares', 'rod_bins'])

print columns_2dcl


#hgcDigis[hgcDigis.layer == 13]

Index([u'n', u'id', u'subdet', u'zside', u'layer', u'wafer', u'wafertype',
       u'cell', u'data', u'pt', u'mipPt', u'energy', u'eta', u'phi', u'x',
       u'y', u'z', u'cluster', u'multicluster'],
      dtype='object')
Index([u'n', u'id', u'mipPt', u'pt', u'energy', u'eta', u'phi', u'layer',
       u'subdet', u'cells', u'multicluster'],
      dtype='object')
['id', 'eta', 'phi', 'x', 'y', 'subdet', 'layer', 'pt', 'energy', 'cells', 'ncells', 'rod_bin_max', 'rod_bin_shares', 'rod_bins']


In [879]:
# these towers are eta-phi....can not be easily compared
#triggerTowers.sort_values(by='energy', axis=0, ascending=False).iloc[:10]

In [880]:
if not tc_rod_bins.empty:
    triggerCells = pd.merge(triggerCells,
                            tc_rod_bins,
                            on='id')

genParticles['pdgid'] = genParticles.pid
# this is not needed anymore in recent versions of the ntuples
# tcsWithPos = pd.merge(triggerCells, tc_geom_df[['id', 'x', 'y']], on='id')
triggerClusters['ncells'] = [len(x) for x in triggerClusters.cells]
if 'x' not in triggerClusters.columns:
    triggerClusters = pd.merge(triggerClusters, tc_geom_df[['z', 'id']], on='id')
    triggerClusters['R'] = triggerClusters.z/np.sinh(triggerClusters.eta)
    triggerClusters['x'] = triggerClusters.R*np.cos(triggerClusters.phi)
    triggerClusters['y'] = triggerClusters.R*np.sin(triggerClusters.phi)

trigger3DClusters['nclu'] = [len(x) for x in trigger3DClusters.clusters]

In [905]:

triggerCells[triggerCells.zside > 0 ][columns_tc]

,id,eta,phi,x,y,subdet,layer,pt,energy,mipPt,cluster,zside
24,1738292527,2.207990,2.658944,-66.910461,35.059860,3,19,0.275722,1.269333,13.800637,1738292527,1
25,1739341099,2.190279,2.643406,-68.035004,37.007629,3,21,0.332862,1.506133,13.310580,1739341099,1
26,1739341103,2.214640,2.658944,-66.910461,35.059860,3,21,0.388820,1.801659,15.548263,1739341099,1
27,1740389648,2.208210,2.680428,-68.597275,34.085976,3,23,0.224938,1.035764,8.994897,1740389648,1
28,1740389675,2.197445,2.643406,-68.035004,37.007629,3,23,0.409999,1.868176,16.395155,1740389648,1
29,1741438224,2.215330,2.680428,-68.597275,34.085976,3,25,0.063325,0.293625,2.532266,1741438225,1
30,1741438225,2.185751,2.660734,-70.002960,36.520687,3,25,0.140273,0.631911,5.609282,1741438225,1
31,1741438251,2.204561,2.643406,-68.035004,37.007629,3,25,0.124468,0.571095,4.977267,1741438225,1
32,1741438255,2.228939,2.658944,-66.910461,35.059860,3,25,0.118507,0.556842,4.738879,1741438225,1
37,1762409749,2.187830,2.672198,-72.195824,36.618080,4,29,0.106470,0.480607,2.112900,0,1


In [882]:
dbsClusters = pd.DataFrame()
for zside in [-1,1]:
    for layer in range (1, 53):
        # print layer
        dbsClusters = dbsClusters.append(buildDBSCANClusters(layer, zside, triggerCells[(triggerCells.zside == zside) & (triggerCells.layer == layer)]), ignore_index=True)

In [883]:
computeClusterRodSharing(dbsClusters, triggerCells)
computeClusterRodSharing(triggerClusters, triggerCells)


In [884]:
seeds = getClusterSeeds(triggerClusters[(triggerClusters.eta > 0)])
new3Dclusters = pd.DataFrame()
for seed in seeds:
    print seed
    components = getClusterComponents(seed, triggerClusters[(triggerClusters.eta > 0)])
    new3Dclusters = new3Dclusters.append(build3D(components), ignore_index=True)


(21, 41)
(21, 41)
[(20, 40), (20, 41), (20, 42), (21, 40), (21, 41), (21, 42), (22, 40), (22, 41), (22, 42)]


In [885]:
seeds = getClusterSeeds(triggerClusters[(triggerClusters.eta < 0)])
for seed in seeds:
    components = getClusterComponents(seed, triggerClusters[(triggerClusters.eta < 0)])
    new3Dclusters = new3Dclusters.append(build3D(components), ignore_index=True)


(46, 28)
[(45, 27), (45, 28), (45, 29), (46, 27), (46, 28), (46, 29), (47, 27), (47, 28), (47, 29)]
(40, 18)
[(39, 17), (39, 18), (39, 19), (40, 17), (40, 18), (40, 19), (41, 17), (41, 18), (41, 19)]
(48, 29)
[(47, 28), (47, 29), (47, 30), (48, 28), (48, 29), (48, 30), (49, 28), (49, 29), (49, 30)]
(51, 40)
[(50, 39), (50, 40), (50, 41), (51, 39), (51, 40), (51, 41), (52, 39), (52, 40), (52, 41)]


In [886]:
dbsseeds = getClusterSeeds(dbsClusters[(dbsClusters.eta > 0)])
new3DDBSclusters = pd.DataFrame()
for seed in dbsseeds:
    print seed
    components = getClusterComponents(seed, dbsClusters[(dbsClusters.eta > 0)])
    new3DDBSclusters = new3DDBSclusters.append(build3D(components), ignore_index=True)


(21, 41)
(21, 41)
[(20, 40), (20, 41), (20, 42), (21, 40), (21, 41), (21, 42), (22, 40), (22, 41), (22, 42)]
(30, 41)
(30, 41)
[(29, 40), (29, 41), (29, 42), (30, 40), (30, 41), (30, 42), (31, 40), (31, 41), (31, 42)]


In [887]:
dbsseeds = getClusterSeeds(dbsClusters[(dbsClusters.eta  < 0)])

for seed in dbsseeds:
    print seed
    components = getClusterComponents(seed, dbsClusters[(dbsClusters.eta < 0)])
    new3DDBSclusters = new3DDBSclusters.append(build3D(components), ignore_index=True)


(46, 28)
(46, 28)
[(45, 27), (45, 28), (45, 29), (46, 27), (46, 28), (46, 29), (47, 27), (47, 28), (47, 29)]
(40, 18)
(40, 18)
[(39, 17), (39, 18), (39, 19), (40, 17), (40, 18), (40, 19), (41, 17), (41, 18), (41, 19)]
(48, 29)
(48, 29)
[(47, 28), (47, 29), (47, 30), (48, 28), (48, 29), (48, 30), (49, 28), (49, 29), (49, 30)]
(44, 26)
(44, 26)
[(43, 25), (43, 26), (43, 27), (44, 25), (44, 26), (44, 27), (45, 25), (45, 26), (45, 27)]
(51, 40)
(51, 40)
[(50, 39), (50, 40), (50, 41), (51, 39), (51, 40), (51, 41), (52, 39), (52, 40), (52, 41)]


In [888]:
new3DClDBetaphi = build3DClustersEtaPhi(dbsClusters)


In [889]:
triggerClusters[(triggerClusters.eta > 0)][columns_2dcl].sort_values(by='layer',ascending=True)

,id,eta,phi,x,y,subdet,layer,pt,energy,cells,ncells,rod_bin_max,rod_bin_shares,rod_bins
7,1738292527,2.207990,2.658944,-66.910454,35.059864,3,19,0.231830,1.067268,[1738292527],1,"(21, 41)","[0.0, 0.0, 0.0, 0.0, 1.26933, 0.0, 0.0, 0.0, 0.0]","[(21, 41)]"
8,1739341099,2.203505,2.651778,-67.422173,35.946159,3,21,0.687952,3.153267,"[1739341099, 1739341103]",2,"(21, 41)","[0.0, 0.0, 0.0, 0.0, 3.30779, 0.0, 0.0, 0.0, 0.0]","[(21, 41)]"
9,1740389648,2.201425,2.656522,-68.235611,35.965179,3,23,0.764611,3.497534,"[1740389648, 1740389675]",2,"(21, 41)","[0.0, 1.03576, 0.0, 0.0, 1.86818, 0.0, 0.0, 0....","[(21, 40), (21, 41)]"
10,1741438225,2.206851,2.658222,-68.415146,35.911270,3,25,0.490412,2.255183,"[1741438225, 1741438224, 1741438251, 1741438255]",4,"(21, 41)","[0.0, 0.293625, 0.0, 0.0, 1.75985, 0.0, 0.0, 0...","[(21, 40), (21, 41)]"


In [890]:
triggerClusters[(triggerClusters.eta > 0)][['pt', 'energy', 'ncells', 'rod_bin_max']].groupby('rod_bin_max').sum()

,pt,energy,ncells
rod_bin_max,,,
"(21, 41)",2.174804,9.973252,9


In [891]:
dbsClusters[dbsClusters.eta > 0][columns_2dcl].sort_values(by='layer',ascending=True)

,id,eta,phi,x,y,subdet,layer,pt,energy,cells,ncells,rod_bin_max,rod_bin_shares,rod_bins
20,1738292527,2.207990,2.658944,-66.910461,35.059860,3,19,0.275722,1.269333,[1738292527],1,"(21, 41)","[0.0, 0.0, 0.0, 0.0, 1.26933, 0.0, 0.0, 0.0, 0.0]","[(21, 41)]"
21,1739341099,2.203498,2.651773,-67.422493,35.946735,3,21,0.721669,3.307792,"[1739341099, 1739341103]",2,"(21, 41)","[0.0, 0.0, 0.0, 0.0, 3.30779, 0.0, 0.0, 0.0, 0.0]","[(21, 41)]"
22,1740389648,2.208210,2.680428,-68.597275,34.085976,3,23,0.224938,1.035764,[1740389648],1,"(21, 40)","[0.0, 0.0, 0.0, 0.0, 1.03576, 0.0, 0.0, 0.0, 0.0]","[(21, 40)]"
23,1740389675,2.197445,2.643406,-68.035004,37.007629,3,23,0.409999,1.868176,[1740389675],1,"(21, 41)","[0.0, 0.0, 0.0, 0.0, 1.86818, 0.0, 0.0, 0.0, 0.0]","[(21, 41)]"
24,1741438224,2.206837,2.658221,-68.416054,35.911842,3,25,0.446554,2.053473,"[1741438224, 1741438225, 1741438251, 1741438255]",4,"(21, 41)","[0.0, 0.293625, 0.0, 0.0, 1.75985, 0.0, 0.0, 0...","[(21, 40), (21, 41)]"
25,1762409749,2.187830,2.672198,-72.195824,36.618080,4,29,0.106470,0.480607,[1762409749],1,"(21, 41)","[0.0, 0.0, 0.0, 0.0, 0.480607, 0.0, 0.0, 0.0, ...","[(21, 41)]"
26,1762934035,2.178680,2.631578,-72.252052,40.416229,4,30,0.204028,0.912805,[1762934035],1,"(21, 41)","[0.0, 0.0, 0.0, 0.0, 0.912805, 0.0, 0.0, 0.0, ...","[(21, 41)]"
27,1764496670,2.847551,2.072574,-20.991514,38.262856,4,33,0.142813,1.235584,[1764496670],1,"(30, 41)","[0.0, 0.0, 0.0, 0.0, 1.23558, 0.0, 0.0, 0.0, 0.0]","[(30, 41)]"
28,1766079744,2.174564,2.649142,-78.957680,42.363998,4,36,0.104371,0.465079,[1766079744],1,"(21, 41)","[0.0, 0.0, 0.0, 0.0, 0.465079, 0.0, 0.0, 0.0, ...","[(21, 41)]"


In [892]:
dbsClusters[(dbsClusters.eta > 0)][['pt', 'energy', 'ncells', 'rod_bin_max']].groupby('rod_bin_max').sum()

,pt,energy,ncells
rod_bin_max,,,
"(21, 40)",0.224938,1.035764,1
"(21, 41)",2.268814,10.357265,11
"(30, 41)",0.142813,1.235584,1


In [893]:
triggerClusters[(triggerClusters.eta < 0)][columns_2dcl].sort_values(by='layer',ascending=True)

,id,eta,phi,x,y,subdet,layer,pt,energy,cells,ncells,rod_bin_max,rod_bin_shares,rod_bins
0,1717327375,-2.174995,-0.457080,68.316139,-33.599022,3,11,0.501131,2.233978,[1717327375],1,"(47, 27)","[0.0, 0.0, 0.0, 0.0, 2.2324, 0.0, 0.0, 0.0, 0.0]","[(47, 27)]"
1,1719434778,-1.977063,0.322580,89.738716,29.995659,3,15,0.090922,0.334593,[1719434778],1,"(51, 40)","[0.0, 0.0, 0.0, 0.0, 0.367482, 0.0, 0.0, 0.0, ...","[(51, 40)]"
2,1719417092,-2.283636,-0.420665,63.000782,-28.184656,3,15,0.261332,1.295450,"[1719417092, 1719424813, 1719417096]",3,"(46, 29)","[0.0, 0.490211, 0.0, 0.0, 0.932458, 0.0, 0.0, ...","[(46, 28), (46, 29)]"
3,1720465668,-2.269636,-0.439481,63.770958,-29.981716,3,17,0.225484,1.102532,"[1720465668, 1720473390, 1720465665]",3,"(46, 28)","[0.0, 0.0, 0.0, 0.0, 0.713803, 0.270145, 0.0, ...","[(46, 28), (47, 28)]"
4,1721521965,-2.272436,-0.406302,64.962769,-27.949675,3,19,0.160140,0.785170,"[1721521965, 1721521966]",2,"(46, 29)","[0.0, 0.0, 0.416785, 0.0, 0.516995, 0.0, 0.0, ...","[(46, 29), (47, 28)]"
5,1722570518,-2.185315,-0.365018,72.715561,-27.787720,3,21,0.193721,0.872315,"[1722570518, 1722570514]",2,"(48, 29)","[0.0, 0.0, 0.0, 0.0, 0.915083, 0.0, 0.0, 0.0, ...","[(48, 29)]"
6,1722562820,-2.288579,-0.436430,63.474361,-29.606102,3,21,0.143298,0.713793,[1722562820],1,"(46, 28)","[0.0, 0.0, 0.0, 0.0, 0.748791, 0.0, 0.0, 0.0, ...","[(46, 28)]"
11,1745631492,-2.329483,-0.436430,63.474339,-29.606085,4,29,0.600736,3.114807,[1745631492],1,"(46, 28)","[0.0, 0.0, 0.0, 0.0, 1.99136, 0.0, 0.0, 0.0, 0.0]","[(46, 28)]"
12,1751436303,-2.046001,-1.194891,39.359093,-99.725830,4,40,0.907000,3.567299,[1751436303],1,"(40, 18)","[0.0, 0.0, 0.0, 0.0, 3.00769, 0.0, 0.0, 0.0, 0.0]","[(40, 18)]"


In [894]:
triggerClusters[(triggerClusters.eta < 0)][['pt', 'energy', 'ncells', 'rod_bin_max']].groupby('rod_bin_max').sum()

,pt,energy,ncells
rod_bin_max,,,
"(40, 18)",0.907000,3.567299,1
"(46, 28)",0.969519,4.931132,5
"(46, 29)",0.421472,2.080620,5
"(47, 27)",0.501131,2.233978,1
"(48, 29)",0.193721,0.872315,2
"(51, 40)",0.090922,0.334593,1


In [895]:
dbsClusters[dbsClusters.eta < 0][columns_2dcl].sort_values(by='layer',ascending=True)

,id,eta,phi,x,y,subdet,layer,pt,energy,cells,ncells,rod_bin_max,rod_bin_shares,rod_bins
0,1717327375,-2.174995,-0.457080,68.316139,-33.599026,3,11,0.500776,2.232399,[1717327375],1,"(47, 27)","[0.0, 0.0, 0.0, 0.0, 2.2324, 0.0, 0.0, 0.0, 0.0]","[(47, 27)]"
1,1719434778,-1.977063,0.322580,89.738731,29.995661,3,15,0.099859,0.367482,[1719434778],1,"(51, 40)","[0.0, 0.0, 0.0, 0.0, 0.367482, 0.0, 0.0, 0.0, ...","[(51, 40)]"
2,1719417364,-2.395327,-0.398086,56.778301,-23.877493,3,15,0.052408,0.289894,[1719417364],1,"(45, 29)","[0.0, 0.0, 0.0, 0.0, 0.289894, 0.0, 0.0, 0.0, ...","[(45, 29)]"
3,1719424771,-2.195159,-0.321293,71.689781,-23.860176,3,15,0.040647,0.184799,[1719424771],1,"(48, 29)","[0.0, 0.0, 0.0, 0.0, 0.184799, 0.0, 0.0, 0.0, ...","[(48, 29)]"
4,1719424813,-2.283629,-0.420668,63.001125,-28.185038,3,15,0.286998,1.422669,"[1719424813, 1719417092, 1719417096]",3,"(46, 29)","[0.0, 0.490211, 0.0, 0.0, 0.932458, 0.0, 0.0, ...","[(46, 28), (46, 29)]"
5,1720473371,-2.110035,-0.342534,78.176460,-27.876989,3,17,0.221299,0.926110,"[1720473371, 1720473374, 1720482340]",3,"(49, 29)","[0.0, 0.381589, 0.0, 0.0, 0.544522, 0.0, 0.0, ...","[(49, 28), (49, 29)]"
6,1720473390,-2.269629,-0.439473,63.771687,-29.981386,3,17,0.201234,0.983948,"[1720473390, 1720465665, 1720465668]",3,"(46, 28)","[0.0, 0.0, 0.0, 0.0, 0.713803, 0.270145, 0.0, ...","[(46, 28), (47, 28)]"
11,1721514246,-2.346544,-0.466974,58.551331,-29.519535,3,19,0.050107,0.264191,[1721514246],1,"(45, 28)","[0.0, 0.0, 0.0, 0.0, 0.264191, 0.0, 0.0, 0.0, ...","[(45, 28)]"
10,1721521966,-2.255130,-0.417950,65.785919,-29.216545,3,19,0.086457,0.416785,[1721521966],1,"(47, 28)","[0.0, 0.0, 0.0, 0.0, 0.416785, 0.0, 0.0, 0.0, ...","[(47, 28)]"
9,1721521965,-2.286503,-0.396619,64.300049,-26.929722,3,19,0.104001,0.516995,[1721521965],1,"(46, 29)","[0.0, 0.0, 0.0, 0.0, 0.516995, 0.0, 0.0, 0.0, ...","[(46, 29)]"


In [896]:
dbsClusters[(dbsClusters.eta < 0)][['pt', 'energy', 'ncells', 'rod_bin_max']].groupby('rod_bin_max').sum()

,pt,energy,ncells
rod_bin_max,,,
"(40, 18)",0.764718,3.007695,1
"(44, 26)",0.113644,0.590026,1
"(45, 28)",0.256564,1.367887,2
"(45, 29)",0.052408,0.289894,1
"(46, 28)",0.819537,4.145055,6
"(46, 29)",0.390999,1.939664,4
"(47, 27)",0.500776,2.232399,1
"(47, 28)",0.086457,0.416785,1
"(47, 30)",0.159307,0.758375,2


In [897]:
trigger3DClusters

,n,id,pt,energy,eta,phi,clusters,showerlength,coreshowerlength,firstlayer,...,seetot,seemax,spptot,sppmax,szz,srrtot,srrmax,srrmean,emaxe,nclu
0,NaN,1717327375,0.501131,2.233977,-2.174995,-0.457080,[1717327375],1,1,11,...,2.384186e-07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1
1,NaN,1719417092,0.790255,3.896011,-2.278052,-0.424548,"[1719417092, 1720465668, 1721521965, 1722562820]",7,4,15,...,1.441091e-02,0.015596,0.018391,0.019102,2.475754,0.003060,0.003325,0.002622,0.332507,4
2,NaN,1738292527,2.174804,9.974965,2.204185,2.655608,"[1738292527, 1739341099, 1740389648, 1741438225]",7,4,19,...,1.122179e-02,0.016764,0.012389,0.017734,2.395228,0.002566,0.003826,0.002503,0.350631,4
3,NaN,1745631492,0.600736,3.114807,-2.329483,-0.436430,[1745631492],1,1,29,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1
4,NaN,1751436303,0.907000,3.567299,-2.046001,-1.194891,[1751436303],1,1,40,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1


In [898]:
new3Dclusters

,energy,eta,phi,pt,layers,clusters,nclu,firstlayer,showerlength,seetot,seemax,spptot,sppmax,szz,emaxe,id
0,10.811005,2.204012,2.655665,2.357481,"[19, 21, 23, 25]","[1738292527, 1739341099, 1740389648, 1741438225]",4,19,1,1,1,1,1,1,1,1738292527
1,10.022371,-2.270592,-0.437016,2.047809,"[11, 15, 17, 19, 21, 29]","[1717327375, 1719417092, 1720465668, 172152196...",6,11,1,1,1,1,1,1,1,1717327375
2,3.866953,-2.046001,-1.194891,0.983188,[40],[1751436303],1,40,1,1,1,1,1,1,1,1751436303
3,0.945589,-2.185315,-0.365018,0.209993,[21],[1722570518],1,21,1,1,1,1,1,1,1,1722570518
4,0.362699,-1.977063,0.322580,0.098560,[15],[1719434778],1,15,1,1,1,1,1,1,1,1719434778


In [899]:
new3DDBSclusters

,energy,eta,phi,pt,layers,clusters,nclu,firstlayer,showerlength,seetot,seemax,spptot,sppmax,szz,emaxe,id
0,12.350043,2.200206,2.654104,2.703111,"[19, 21, 23, 23, 25, 29, 30, 36]","[1738292527, 1739341099, 1740389648, 174038967...",8,19,1,1,1,1,1,1,1,1738292527
1,1.339373,2.847551,2.072574,0.154810,[33],[1764496670],1,33,1,1,1,1,1,1,1,1764496670
2,11.264585,-2.280493,-0.445239,2.279418,"[11, 15, 15, 17, 19, 19, 19, 21, 23, 29, 33]","[1717327375, 1719417364, 1719424813, 172047339...",11,11,1,1,1,1,1,1,1,1717327375
3,3.260341,-2.046001,-1.194891,0.828955,[40],[1751436303],1,40,1,1,1,1,1,1,1,1751436303
4,3.848602,-2.180376,-0.344162,0.858811,"[15, 17, 19, 19, 19, 21, 21]","[1719424771, 1720473371, 1721521920, 172152195...",7,15,1,1,1,1,1,1,1,1719424771
5,0.639588,-2.330836,-0.655464,0.123190,[32],[1747211530],1,32,1,1,1,1,1,1,1,1747211530
6,0.398350,-1.977063,0.322580,0.108248,[15],[1719434778],1,15,1,1,1,1,1,1,1,1719434778


In [900]:
new3DClDBetaphi

,energy,eta,phi,pt,layers,clusters,nclu,firstlayer,showerlength,seetot,seemax,spptot,sppmax,szz,emaxe,id
0,4.888997,-2.280226,-0.425850,0.989560,"[15, 17, 19, 19, 21, 23]","[1719424813, 1720473390, 1721521965, 172152196...",6,15,1,1,1,1,1,1,1,1719424813
1,3.260341,-2.046001,-1.194891,0.828955,[40],[1751436303],1,40,1,1,1,1,1,1,1,1751436303
2,12.350043,2.200206,2.654104,2.703111,"[19, 21, 23, 23, 25, 29, 30, 36]","[1738292527, 1739341099, 1740389648, 174038967...",8,19,1,1,1,1,1,1,1,1738292527


In [901]:
# tcs_in_3Dcl = np.concatenate(triggerClusters.cells)
# tcs_in_3Dcl

In [902]:
# triggerCells['used_3D'] = triggerCells.id.isin(tcs_in_3Dcl)


In [903]:
# triggerCells[['used_3D', 'id']]

In [904]:


# triggerCells[~triggerCells.used_3D]